In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter
%load_ext autoreload
%autoreload 2
from alexnet import AlexNet,hash_loss

import torch.optim as optim

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
BATCH_SIZE = 128

download_data = False
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=download_data,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=download_data, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def get_accuracy(labels,logits,reduce_mean):
    acc = (labels == logits).sum()
    if reduce_mean:
        acc = acc/logtits.size()[0]
    return acc
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
CUDA =True

model = AlexNet()
if CUDA:
    model.cuda()
optimizer = torch.optim.Adam(model.parameters())
writer = SummaryWriter()

In [ ]:
N_EPOCHS = 5
for epoch in range(1,N_EPOCHS+1):  # loop over the dataset multiple times

    running_loss = 0.0
    running_acc = 0.0
    running_acc_hash = 0.0
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        preds,preds_hash,ps,qs = model(inputs)
        _,logits = torch.max(preds,1)
        _,logits_hash = torch.max(preds_hash,1) 
        
        
        loss = hash_loss(labels,preds,preds_hash,ps,qs)
        loss.backward()
        optimizer.step()
        
        acc = get_accuracy(labels,logits,reduce_mean = False)
        acc_hash = get_accuracy(labels,logits_hash,reduce_mean = False)
        
        n=i*BATCH_SIZE
        running_loss += loss.item()
        running_acc = (running_acc*n + acc)/(n+1)
        running_acc_hash = (running_acc_hash*n + acc_hash)/(n+1)
        
        
        writer.add_scalar('Loss/train', running_loss , (i+1)*epoch)
        writer.add_scalar('Accuracy/train_original', acc , (i+1)*epoch)
        writer.add_scalar('Accuracy/train_hashed', acc_hash , (i+1)*epoch)
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')